<a href="https://colab.research.google.com/github/03sarath/Weights-and-Biases-/blob/master/W%26B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Install & Setup


In [ ]:
!pip install --upgrade pip
!pip install tensorflow scikit-learn wandb

# Install necessary libraries

In [ ]:
import os
import wandb
from wandb.integration.keras import (
    WandbMetricsLogger,
    WandbModelCheckpoint,
    # WandbEvalCallback  # Uncomment if you want to log sample predictions
)
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

# 2. Initialize W&B


In [ ]:
wandb.login()

In [ ]:
run = wandb.init(
    project="sample-test",
    config={
        "learning_rate": 0.01,
        "epochs": 20,
        "batch_size": 16,
        "architecture": "Sequential",
        "dataset": "Iris"
    }
)
config = run.config

# 3. Load & Prepare Data

In [ ]:
data = load_iris()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. Build the Model

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(4,)),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(16, activation="relu"),
    keras.layers.Dense(3, activation="softmax")
])
model.compile(
    optimizer=keras.optimizers.Adam(config.learning_rate),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

# 5. Prepare Callbacks

In [ ]:
# Ensure the models directory exists
os.makedirs("models", exist_ok=True)

callbacks = [
    # Log epoch metrics and resource usage
    WandbMetricsLogger(log_freq="epoch"),

    # Save every epoch as a .keras file so W&B can always find it
    WandbModelCheckpoint(
        filepath="models/iris_model_{epoch:02d}.keras",
        save_freq="epoch",         # write a file each epoch
        save_best_only=False,      # ensure every epoch is saved
        save_weights_only=False    # full SavedModel (.keras) format
    ),
]

# 6. Train

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=int(config.epochs),
    batch_size=int(config.batch_size),
    callbacks=callbacks
)

# 7. Evaluate & Finish

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {loss:.4f}  |  Test Accuracy: {acc:.4f}")
run.finish()